# whisperのインストール
!pip install git+https://github.com/openai/whisper.git

In [1]:
# 冒頭5分だけにする
from pydub import AudioSegment

audio = AudioSegment.from_wav("sample.wav")
five_minutes = 5 * 60 * 1000  
audio_cut = audio[:five_minutes]

audio_cut.export("sample.wav", format="wav")


冒頭5分を 'sample_cut.wav' に保存しました。


In [38]:
!pip install pydub
!pip install -q pyannote.audio
!pip install -Uq webvtt-py

In [32]:
# Hugginface Hubにログイン
from huggingface_hub import notebook_login
notebook_login()

In [2]:
# Cudaを利用するための設定
from pyannote.audio import Pipeline
import torchaudio

pipeline = Pipeline.from_pretrained(
  "pyannote/speaker-diarization-3.1",
  use_auth_token=True)

import torch
pipeline.to(torch.device("cuda"))

In [16]:
# 話者分離
diarization = pipeline("sample_cut.wav", num_speakers=2)

with open("diarization.txt", "w") as text_file:
    text_file.write(str(diarization))

print(*list(diarization.itertracks(yield_label = True))[:10], sep="\n")

(<Segment(0.655344, 5.26222)>, 'A', 'SPEAKER_00')
(<Segment(5.26222, 6.93284)>, 'B', 'SPEAKER_01')
(<Segment(7.16909, 9.29534)>, 'C', 'SPEAKER_00')
(<Segment(10.0378, 15.5053)>, 'D', 'SPEAKER_00')
(<Segment(15.5391, 16.9397)>, 'E', 'SPEAKER_01')
(<Segment(17.4966, 18.6441)>, 'F', 'SPEAKER_00')
(<Segment(19.7916, 20.7366)>, 'G', 'SPEAKER_00')
(<Segment(22.1203, 22.3903)>, 'H', 'SPEAKER_00')
(<Segment(23.4535, 26.5585)>, 'I', 'SPEAKER_00')
(<Segment(25.9678, 27.5035)>, 'J', 'SPEAKER_01')


In [6]:
# グルーピング
import re

def millisec(timeStr):
  spl = timeStr.split(":")
  s = (int)((int(spl[0]) * 60 * 60 + int(spl[1]) * 60 + float(spl[2]) )* 1000)
  return s


dzs = open('diarization.txt').read().splitlines()

groups = []
g = []
lastend = 0

for d in dzs:
  if g and (g[0].split()[-1] != d.split()[-1]):
    groups.append(g)
    g = []

  g.append(d)

  end = re.findall('[0-9]+:[0-9]+:[0-9]+\.[0-9]+', string=d)[1]
  end = millisec(end)
  if (lastend > end):
    groups.append(g)
    g = []
  else:
    lastend = end
if g:
  groups.append(g)
print(*groups, sep='\n')

['[ 00:00:00.655 -->  00:00:05.262] A SPEAKER_00']
['[ 00:00:05.262 -->  00:00:06.932] B SPEAKER_01']
['[ 00:00:07.169 -->  00:00:09.295] C SPEAKER_00', '[ 00:00:10.037 -->  00:00:15.505] D SPEAKER_00']
['[ 00:00:15.539 -->  00:00:16.939] E SPEAKER_01']
['[ 00:00:17.496 -->  00:00:18.644] F SPEAKER_00', '[ 00:00:19.791 -->  00:00:20.736] G SPEAKER_00', '[ 00:00:22.120 -->  00:00:22.390] H SPEAKER_00', '[ 00:00:23.453 -->  00:00:26.558] I SPEAKER_00']
['[ 00:00:25.967 -->  00:00:27.503] J SPEAKER_01']
['[ 00:00:27.857 -->  00:00:29.342] K SPEAKER_00', '[ 00:00:29.562 -->  00:00:30.625] L SPEAKER_00', '[ 00:00:31.806 -->  00:00:32.295] M SPEAKER_00', '[ 00:00:33.814 -->  00:00:38.657] N SPEAKER_00']
['[ 00:00:38.489 -->  00:00:39.484] O SPEAKER_01']
['[ 00:00:39.805 -->  00:00:40.699] P SPEAKER_00', '[ 00:00:41.796 -->  00:00:41.931] Q SPEAKER_00', '[ 00:00:42.150 -->  00:00:43.197] R SPEAKER_00', '[ 00:00:44.226 -->  00:00:46.589] S SPEAKER_00']
['[ 00:00:47.129 -->  00:00:51.179] T SPE

In [26]:
# 音声分離
import os
import re
from pydub import AudioSegment

# フォルダが存在しない場合は作成
output_dir = "voice"
os.makedirs(output_dir, exist_ok=True)

# オーディオファイルを読み込む
audio = AudioSegment.from_wav("sample_cut.wav")

gidx = -1
for g in groups:
    start = re.findall(r'[0-9]+:[0-9]+:[0-9]+\.[0-9]+', string=g[0])[0]
    end = re.findall(r'[0-9]+:[0-9]+:[0-9]+\.[0-9]+', string=g[-1])[1]

    start = millisec(start)
    end = millisec(end) 

    print(start, end)

    gidx += 1
    audio[start:end].export(os.path.join(output_dir, f"{gidx}.wav"), format='wav')


655 5262
5262 6932
7169 15505
15539 16939
17496 26558
25967 27503
27857 38657
38489 39484
39805 46589
47129 55802
55802 56545
56545 56832
56562 56579
57321 59684
60696 65624
65708 69201
69690 105854
105515 105920
105920 110258
110258 115455
115911 121615
121615 139334
134254 134423
139772 151163
149374 149391
151264 153745
153711 153947
153947 155871
153981 154082
156023 160799
160815 169320
169439 176880
175733 175969
176678 180525
177454 177539
180846 195409
194262 195004
195578 195612
195612 201197
200404 200674
201990 208352
208352 211322
212554 220755
221970 231404
231758 234610
234880 235065
236382 240567
240550 245359
246456 251974
251924 261509
261829 270486
270250 282602
282602 282737
282737 282788
285505 289487
290129 290399
290753 298988


In [3]:
# 作業フォルダを作成しWhisperを実行
import os

output_dir = "temp"
os.makedirs(output_dir, exist_ok=True)

for i in range(gidx + 1):
    input_wav_file = os.path.join("voice", f"{i}.wav")
    !whisper {input_wav_file} --language Japanese --model turbo --output_dir {output_dir} --device cuda

In [10]:
# 文字起こし結果をマージ
import os
import re
import webvtt

from datetime import timedelta

gidx = -1
temp_dir = "temp"  
spacermilli = 2000
output_file = "captions_output.txt"  

with open(output_file, "w", encoding="utf-8") as f_out:
    for g in groups:
        shift = re.findall('[0-9]+:[0-9]+:[0-9]+\.[0-9]+', string=g[0])[0]
        shift = millisec(shift) - spacermilli 
        shift = max(shift, 0)

        gidx += 1
        vtt_file = os.path.join(temp_dir, f"{gidx}.vtt")
        
        captions = [[(int)(millisec(caption.start)), (int)(millisec(caption.end)), caption.text]
                    for caption in webvtt.read(vtt_file)]
        
        if captions:
            speaker = g[0].split()[-1]
            f_out.write(f"Speaker: {speaker}\n")  

            for c in captions:
                start = shift + c[0]
                start = start / 1000.0  
                startStr = '{0:02d}:{1:02d}:{2:06.3f}'.format(int(start // 3600),
                                                              int(start % 3600 // 60),
                                                              start % 60)

                end = shift + c[1]
                end = end / 1000.0  
                endStr = '{0:02d}:{1:02d}:{2:06.3f}'.format(int(end // 3600),
                                                            int(end % 3600 // 60),
                                                            end % 60)

                f_out.write(f'[{startStr} --> {endStr}] [{speaker}] {c[2]}\n')
            
            f_out.write("=" * 50 + "\n") 

print(f"結果が {output_file} に保存されました。")


結果が captions_output.txt に保存されました。


# 途中実行時用のコード

In [2]:
# 途中実行用 (gidxの初期化)
import os
import re

output_dir = "temp"
os.makedirs(output_dir, exist_ok=True)

voice_dir = "voice"
existing_files = [f for f in os.listdir(voice_dir) if f.endswith('.wav')]

if existing_files:
    existing_indices = [int(re.findall(r'\d+', f)[0]) for f in existing_files if re.findall(r'\d+', f)]
    max_gidx = max(existing_indices)  
else:
    max_gidx = -1 

gidx = max_gidx